<a href="https://colab.research.google.com/github/xrobin/moving_beyond_memorisation/blob/main/2_Scoring_with_OpenStructure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run this cell, wait for the kernel to restart and then run the next cell:

# Setup

In [2]:
!pip install -q git+https://github.com/conda-incubator/condacolab.git@0.1.x
import condacolab
condacolab.install()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [5]:
!mamba install -q pip scipy networkx conda-forge::boost aivant::openstructure anaconda::py-boost

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [ ]:
!ost --version

In [ ]:
import ost
print(ost.__version__)

In [ ]:
# Check conop
from ost import conop
print(conop.GetDefaultLib().FindCompound("A1LU6").name)

# Protein-protein complex scoring

## Data loading

In [ ]:
!mkdir -p T1187/
!wget https://raw.githubusercontent.com/xrobin/moving_beyond_memorisation/refs/heads/main/2_Scoring_with_OpenStructure_Data/ppi_scoring/T1187o.pdb -O T1187/T1187o.pdb
!wget https://raw.githubusercontent.com/xrobin/moving_beyond_memorisation/refs/heads/main/2_Scoring_with_OpenStructure_Data/ppi_scoring/T1187TS447_1o_superposed.pdb -O T1187/T1187TS447_1o_superposed.pdb
!ls T1187/

In [ ]:
from ost import io
# Target
target_structure = io.LoadPDB("T1187/T1187o.pdb")
# Model
model_structure = io.LoadPDB("T1187/T1187TS447_1o_superposed")

## Scoring with Python interactively

In [ ]:


from ost.mol.alg import scoring

# The Scorer object processes the input structures and performs basic cleanup.
scorer = scoring.Scorer(model_structure, target_structure)

# HTML documentation available as:
# https://openstructure.org/docs/2.8/mol/alg/scoring/#ost.mol.alg.scoring.Scorer
# raw doc string can be displayed with:
# help(scoring.Scorer)


# Here we only scratch the surface and investigate a couple of relevant scores.
# All scores are lazily evaluated and available as attributes.

# the following scores operate on the full assembly which requires to derive a
# one-to-one correspondance between model and reference chains, aka chain
# mapping - OpenStructure does this fully automatically
print("lDDT", scorer.lddt)
print("lDDT (backbone only):", scorer.bb_lddt)
print("QS-score:", scorer.qs_global)

# here is the used mapping:
print("mapping (keys: trg chain, values: mdl chain):",
      scorer.mapping.GetFlatMapping())

# Pinder strictly operates on dimers which is basically the CAPRI use-case
# This is what DockQ and the 3 underlying scores (fnat, irmsd, lrmsd) are
# designed for. Let's first check the interfaces in our structure:
print("Interfaces evaluated by DockQ:", scorer.dockq_interfaces)
help(scoring.Scorer.dockq_interfaces)
print("With their respective DockQ scores:", scorer.dockq_scores)
print("fnat:", scorer.fnat)
print("irmsd:", scorer.irmsd)
print("lrmsd:", scorer.lrmsd)

## Scoring from the command line


In [ ]:
!ost compare-structures -m T1187/T1187TS447_1o_superposed -mf pdb -r T1187/T1187o.pdb --lddt --bb-lddt --qs-score --dockq --out my_out.json

# Protein-ligand complex scoring

## Data loading

In [6]:
!mkdir -p 9CE4/
!wget https://raw.githubusercontent.com/xrobin/moving_beyond_memorisation/refs/heads/main/2_Scoring_with_OpenStructure_Data/ligand_scoring/9CE4_A.pdb?token=GHSAT0AAAAAACXZFLBPGUJT75QK262ZMAHSZXNOKJQ -O 9CE4/9CE4_A.pdb
!wget https://raw.githubusercontent.com/xrobin/moving_beyond_memorisation/refs/heads/main/2_Scoring_with_OpenStructure_Data/ligand_scoring/9CE4_lig.sdf?token=GHSAT0AAAAAACXZFLBOJFQDOZJKFOJTM4J6ZXNOKBA -O 9CE4/9CE4_lig.sdf
!wget https://raw.githubusercontent.com/xrobin/moving_beyond_memorisation/refs/heads/main/2_Scoring_with_OpenStructure_Data/ligand_scoring/996_model1.pdb?token=GHSAT0AAAAAACXZFLBPHIDZP3GIB5RJ3AREZXNOI6Q -O 9CE4/996_model1.pdb
!wget https://raw.githubusercontent.com/xrobin/moving_beyond_memorisation/refs/heads/main/2_Scoring_with_OpenStructure_Data/ligand_scoring/996_model1_ligand1.sdf?token=GHSAT0AAAAAACXZFLBP5KJ36JXD7NOBDLPSZXNOKPQ -O 9CE4/996_model1_ligand1.sdf
!ls 9CE4/

--2024-09-23 08:41:22--  https://raw.githubusercontent.com/plinder-org/moving_beyond_memorisation/refs/heads/main/2_Scoring_with_OpenStructure_Data/ligand_scoring/9CE4_A.pdb?token=GHSAT0AAAAAACXZFLBPGUJT75QK262ZMAHSZXNOKJQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-09-23 08:41:22 ERROR 404: Not Found.

--2024-09-23 08:41:22--  https://raw.githubusercontent.com/plinder-org/moving_beyond_memorisation/refs/heads/main/2_Scoring_with_OpenStructure_Data/ligand_scoring/9CE4_lig.sdf?token=GHSAT0AAAAAACXZFLBOJFQDOZJKFOJTM4J6ZXNOKBA
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP 

In [ ]:
!ls -l 9CE4/

total 0
-rw-r--r-- 1 root root 0 Sep 20 15:17 996_model1_ligand1.sdf
-rw-r--r-- 1 root root 0 Sep 20 15:17 996_model1.pdb
-rw-r--r-- 1 root root 0 Sep 20 15:17 9CE4_A.pdb
-rw-r--r-- 1 root root 0 Sep 20 15:17 9CE4_lig.sdf


In [ ]:
from ost import io
# Target
target_structure = io.LoadPDB("9CE4/9CE4_A.pdb")
target_ligand = io.LoadSDF("9CE4/9CE4_lig.sdf")
# Model
model_structure = io.LoadPDB("9CE4/996_model1.pdb")
model_ligand = io.LoadSDF("9CE4/996_model1_ligand1.sdf")


OSError: File '9CE4/9CE4_A.pdb' doesn't contain any entities

## Data cleanup



For ligand scoring in Python, structures must be cleaned up hydrogen atoms removed before executing the scorer. Protein structures are cleaned with Molck (the Molecular checker), and ligands with a simple selection.

In [ ]:
# Cleanup a copy of the protein structures
from ost import conop
from ost.mol.alg import Molck, MolckSettings
cleaned_model_structure = model_structure.Copy()
cleaned_target_structure = target_structure.Copy()
molck_settings = MolckSettings(rm_unk_atoms=True,  # Remove unknown atoms
                               rm_non_std=False,  # Keep non standard residues
                               rm_hyd_atoms=True,  # Remove Hydrogens
                               rm_oxt_atoms=False,  # Keep terminal oxygens
                               rm_zero_occ_atoms=False,  # Keep atoms with 0 occupancy
                               colored=False,
                               map_nonstd_res=False,
                               assign_elem=True)
Molck(cleaned_model_structure, conop.GetDefaultLib(), molck_settings)
Molck(cleaned_target_structure, conop.GetDefaultLib(), molck_settings)

In [ ]:
# Cleanup the ligands
# Remove hydrogens
cleaned_model_ligand = model_ligand.Select("ele != H and ele != D")
cleaned_target_ligand = target_ligand.Select("ele != H and ele != D")


In [ ]:
from ost.mol.alg.ligand_scoring import LDDTPLIScorer, SCRMSDScorer

help(LDDTPLIScorer)
help(SCRMSDScorer)

## Scoring with Python interactively

In [ ]:
# Score with LDDT-PLI
scorer = LDDTPLIScorer(
    target = cleaned_target_structure,
    target_ligands = [cleaned_target_ligand],
    model = cleaned_model_structure,
    model_ligands = [cleaned_model_ligand],
    # Extra arguments
    resnum_alignments=True,
    )
chain_name = cleaned_model_ligand.chains[0].name
residue_number = cleaned_model_ligand.residues[0].number
print("LDDT-PLI: ", scorer.score[chain_name][residue_number])

# Score with RMSD
scorer = SCRMSDScorer(
    target = cleaned_target_structure,
    target_ligands = [cleaned_target_ligand],
    model = cleaned_model_structure,
    model_ligands = [cleaned_model_ligand],
    # Extra arguments
    resnum_alignments=True,
    )
print("BiSyRMSD: ", scorer.score[chain_name][residue_number])
print("LDDT-LP: ", scorer.aux[chain_name][residue_number]["lddt_lp"])


LDDT-PLI:  0.7669142
BiSyRMSD:  1.943465
LDDT-LP:  0.9311297811187265


## Scoring from the command line

In [ ]:
! ost compare-ligand-structures \
    --reference 9CE4/9CE4_A.pdb \
    --reference-ligands 9CE4/9CE4_lig.sdf \
    --model 9CE4/996_model1.pdb \
    --model-ligands 9CE4/996_model1_ligand1.sdf \
    --output T1186LG350_1.json \
    --lddt-pli \
    --rmsd